In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from xgboost import XGBClassifier, XGBRegressor
from bayes_opt import BayesianOptimization

In [19]:
features_num = [
    'Total_flux', 'Peak_flux', 
       'NUV_flux_corr', 'u_flux_corr', 'Bw_flux_corr', 'R_flux_corr',
       'I_flux_corr', 'z_flux_corr', 'y_flux_corr',
       'J_flux_corr', 'H_flux_corr', 'K_flux_corr', 'Ks_flux_corr',
       'ch1_flux_corr', 'ch2_flux_corr', 'ch3_flux_corr', 'ch4_flux_corr',
       'F_MIPS_24', 'F_PACS_100', 'F_PACS_160', 'F_SPIRE_250', 'F_SPIRE_350',
       'F_SPIRE_500', 'Z_BEST',
       'g_flux_corr', 'nb921_hsc_flux_corr'
    ]
y_column = "Classification"

classes = ['jet-mode radio AGN/low-excitation radio galaxy', 'quasar-like radio AGN / high-excitation radio galaxy', 
           'radio-quiet AGN', 'star-forming galaxy']

In [20]:
data = pd.read_csv("../../../Data/Fangyou_data/Cleaned/combined_using_similar_columns.csv")

X = data[features_num]
y = data[[y_column, 'Source']]

# 2 class

In [21]:
def AGN(row):
    
    if row['Classification'] == 'jet-mode radio AGN/low-excitation radio galaxy':
        return 1
    elif row['Classification'] == 'quasar-like radio AGN / high-excitation radio galaxy':
        return 1
    elif row['Classification'] == 'radio-quiet AGN':
        return 1
    elif row['Classification'] == 'star-forming galaxy':
        return 0

In [22]:
y['AGN'] =  y.apply(AGN, axis=1, result_type='expand')

/var/lib/condor/execute/dir_830810/ipykernel_873943/3505554675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['AGN'] =  y.apply(AGN, axis=1, result_type='expand')


# Training model

In [147]:
X

,Total_flux,Peak_flux,NUV_flux_corr,u_flux_corr,Bw_flux_corr,R_flux_corr,I_flux_corr,z_flux_corr,y_flux_corr,J_flux_corr,...,ch4_flux_corr,F_MIPS_24,F_PACS_100,F_PACS_160,F_SPIRE_250,F_SPIRE_350,F_SPIRE_500,Z_BEST,g_flux_corr,nb921_hsc_flux_corr
0,841.504227,504.067340,0.121041,NaN,NaN,1.382728,NaN,2.296511,NaN,5.001355,...,64.083221,NaN,NaN,NaN,NaN,NaN,NaN,2.4986,NaN,NaN
1,1497.581978,934.846936,NaN,13.038874,NaN,NaN,0.011940,126.802889,126.964987,186.723870,...,1071.471101,NaN,NaN,NaN,NaN,NaN,NaN,4.5887,NaN,NaN
2,502.877076,322.345615,0.313706,1.890881,NaN,6.857692,18.628942,15.904630,19.680046,27.588542,...,77.374759,292.823334,7114.09200,8275.3570,5009.754395,2178.669678,990.541626,2.7221,NaN,NaN
3,300.955344,329.597236,7.500074,NaN,NaN,63.557111,96.889359,81.025505,NaN,100.024530,...,495.052235,NaN,NaN,NaN,NaN,NaN,NaN,0.1513,NaN,NaN
4,285.588580,318.506481,NaN,0.011021,NaN,NaN,NaN,0.743611,0.080719,0.027066,...,26.426610,303.623596,5778.15500,58864.6880,20822.921875,25510.591797,9542.652344,4.4603,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77604,404.272879,344.389065,0.225079,0.431178,NaN,1.626601,NaN,7.158808,NaN,NaN,...,51.112049,323.144684,565.49347,1275.3830,NaN,NaN,NaN,0.9986,0.609437,NaN
77605,1480.551760,936.415793,0.172858,0.609065,NaN,2.296721,NaN,9.497899,NaN,NaN,...,51.009281,306.243469,563.41090,1235.7626,NaN,NaN,NaN,1.0274,0.949995,NaN
77606,709.620314,374.982368,0.217307,0.979376,NaN,2.811726,NaN,9.116296,NaN,NaN,...,65.887466,305.930634,545.68060,1352.5192,NaN,NaN,NaN,1.0350,1.320066,NaN
77607,591.226106,497.437861,0.366819,0.469435,NaN,5.373043,NaN,15.577719,NaN,NaN,...,31.999180,311.362915,536.16010,1616.1852,NaN,NaN,NaN,0.4429,1.528544,NaN


In [179]:
X_train, X_test, y_train, y_test = train_test_split(X_temp, y[['AGN', 'Source']], train_size=0.8, stratify=y[['AGN', 'Source']])
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.8, stratify=y_test)

In [180]:
model = XGBClassifier(use_label_encoder=False, 
                      max_depth=7,
                     min_child_weight=5,
                     learning_rate=0.2,
                     reg_alpha=5.7,
                     reg_lambda=3.6,
                     tree_method='hist',
                     objective='binary:logistic',
                     #eval_metric=['error'],
                     #scale_pos_weight=frac,
                      nthread=8,
                      n_estimators=10**5,
                      random_state=42
                      )

In [181]:
eval_set = [(X_train, y_train['AGN']), (X_val, y_val['AGN'])]

In [182]:
model.fit(X_train, y_train['AGN'], verbose=True, eval_set=eval_set, early_stopping_rounds=5)

[16:44:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.62457	validation_1-logloss:0.62432
[1]	validation_0-logloss:0.58072	validation_1-logloss:0.58213
[2]	validation_0-logloss:0.54659	validation_1-logloss:0.54880
[3]	validation_0-logloss:0.52192	validation_1-logloss:0.52532
[4]	validation_0-logloss:0.50269	validation_1-logloss:0.50681
[5]	validation_0-logloss:0.48804	validation_1-logloss:0.49286
[6]	validation_0-logloss:0.47623	validation_1-logloss:0.48150
[7]	validation_0-logloss:0.46725	validation_1-logloss:0.47345
[8]	validation_0-logloss:0.45875	validation_1-logloss:0.46692
[9]	validation_0-logloss:0.45105	validation_1-logloss:0.46119
[10]	validation_0-logloss:0.44536	validation_1-logloss:0.45660
[11]	validation_0-logloss:0.44028	validation_1-logloss:0.45234
[1

[126]	validation_0-logloss:0.33614	validation_1-logloss:0.39834
[127]	validation_0-logloss:0.33559	validation_1-logloss:0.39825
[128]	validation_0-logloss:0.33530	validation_1-logloss:0.39830


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=7,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100000, n_jobs=8, nthread=8, num_parallel_tree=1,
              random_state=42, reg_alpha=5.7, reg_lambda=3.6,
              scale_pos_weight=1, subsample=1, tree_method='hist',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [183]:
print(classification_report(model.predict(X_test), y_test['AGN'], digits=4))
print(balanced_accuracy_score(model.predict(X_test), y_test['AGN']))

              precision    recall  f1-score   support

           0     0.9385    0.8305    0.8812     10241
           1     0.4826    0.7440    0.5854      2176

    accuracy                         0.8153     12417
   macro avg     0.7105    0.7873    0.7333     12417
weighted avg     0.8586    0.8153    0.8294     12417

0.7872555197318162


# Reducing features and seeing accuracy

In [148]:
# We remove features in sets, in things that are likely to be there
to_be_removed = [['Total_flux', 'Peak_flux'],
                 ['u_flux_corr'],
                 ['z_flux_corr', 'y_flux_corr', 'I_flux_corr', 'J_flux_corr', 'H_flux_corr', 'K_flux_corr', 'Ks_flux_corr'],
                 ['ch1_flux_corr', 'ch2_flux_corr'], 
                 ['ch3_flux_corr', 'ch4_flux_corr'],
                 ['F_MIPS_24'],
                 ['F_PACS_100', 'F_PACS_160'],
                 ['F_SPIRE_250', 'F_SPIRE_350', 'F_SPIRE_500']
                ]

In [151]:
for c in to_be_removed:
    X_removed = X_test.copy()
    X_removed[c] = np.nan
    print(c, balanced_accuracy_score(model.predict(X_removed), y_test['AGN']))

['Total_flux', 'Peak_flux']


ValueError: feature_names mismatch: ['NUV_flux_corr', 'u_flux_corr', 'Bw_flux_corr', 'R_flux_corr', 'I_flux_corr', 'z_flux_corr', 'y_flux_corr', 'J_flux_corr', 'H_flux_corr', 'K_flux_corr', 'Ks_flux_corr', 'ch1_flux_corr', 'ch2_flux_corr', 'ch3_flux_corr', 'ch4_flux_corr', 'F_MIPS_24', 'F_PACS_100', 'F_PACS_160', 'F_SPIRE_250', 'F_SPIRE_350', 'F_SPIRE_500', 'Z_BEST', 'g_flux_corr', 'nb921_hsc_flux_corr'] ['Total_flux', 'Peak_flux', 'NUV_flux_corr', 'u_flux_corr', 'Bw_flux_corr', 'R_flux_corr', 'I_flux_corr', 'z_flux_corr', 'y_flux_corr', 'J_flux_corr', 'H_flux_corr', 'K_flux_corr', 'Ks_flux_corr', 'ch1_flux_corr', 'ch2_flux_corr', 'ch3_flux_corr', 'ch4_flux_corr', 'F_MIPS_24', 'F_PACS_100', 'F_PACS_160', 'F_SPIRE_250', 'F_SPIRE_350', 'F_SPIRE_500', 'Z_BEST', 'g_flux_corr', 'nb921_hsc_flux_corr']
training data did not have the following fields: Peak_flux, Total_flux

# BH stuff

In [255]:
model = XGBClassifier(use_label_encoder=False, 
                      max_depth=1,
                     #min_child_weight=5,
                     #learning_rate=0.2,
                     #reg_alpha=5.7,
                     #reg_lambda=3.6,
                     tree_method='hist',
                     objective='binary:logistic',
                     eval_metric=['error'],
                     #scale_pos_weight=frac,
                      nthread=8,
                      n_estimators=10**5,
                      random_state=42
                      )

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y[['AGN', 'Source']], train_size=0.8, stratify=y[['AGN', 'Source']])
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.8, stratify=y_test)

In [257]:
eval_set = [(X_train, y_train['AGN']), (Best_Heckman_X, Best_Heckman_y['AGN']), (X_val, y_val['AGN'])]

In [258]:
model.fit(X_train, y_train['AGN'], verbose=True, eval_set=eval_set, early_stopping_rounds=5)

[0]	validation_0-error:0.22132	validation_1-error:0.77868	validation_2-error:0.22576
[1]	validation_0-error:0.24031	validation_1-error:0.77868	validation_2-error:0.24283
[2]	validation_0-error:0.22132	validation_1-error:0.77868	validation_2-error:0.22576
[3]	validation_0-error:0.20861	validation_1-error:0.42554	validation_2-error:0.21256
[4]	validation_0-error:0.20521	validation_1-error:0.42554	validation_2-error:0.20837
[5]	validation_0-error:0.19811	validation_1-error:0.42554	validation_2-error:0.20000
[6]	validation_0-error:0.20983	validation_1-error:0.42554	validation_2-error:0.21385
[7]	validation_0-error:0.19592	validation_1-error:0.42554	validation_2-error:0.19807
[8]	validation_0-error:0.19281	validation_1-error:0.42554	validation_2-error:0.19678
[9]	validation_0-error:0.19281	validation_1-error:0.42554	validation_2-error:0.19678
[10]	validation_0-error:0.18877	validation_1-error:0.41985	validation_2-error:0.19549
[11]	validation_0-error:0.19304	validation_1-error:0.42554	valid

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric=['error'],
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=1, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100000, n_jobs=8,
              nthread=8, num_parallel_tree=1, random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='hist',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [201]:
print(classification_report(model.predict(X_test), y_test['AGN'], digits=4))
print(balanced_accuracy_score(model.predict(X_test), y_test['AGN']))

              precision    recall  f1-score   support

           0     0.9427    0.8657    0.9026      9868
           1     0.6051    0.7964    0.6877      2549

    accuracy                         0.8515     12417
   macro avg     0.7739    0.8311    0.7951     12417
weighted avg     0.8734    0.8515    0.8585     12417

0.8310591729225144


In [280]:
import lightgbm as lgb

param = {'n_estimators': 172,
         'num_leaves': 1011,
         'min_child_samples': 16, 
         'objective': 'binary',
         'log_max_bin': 9,
         'colsample_bytree': 0.46
        }

In [281]:
train_data = lgb.Dataset(X_train, label=y_train['AGN'])
validation_data = lgb.Dataset(X_val, label=y_val['AGN'])
Best_heckman = lgb.Dataset(Best_Heckman_X, label=Best_Heckman_y['AGN'])

In [282]:
bst = lgb.train(param, train_data, 200, valid_sets=[validation_data], early_stopping_rounds=5)

[LightGBM] [Warning] Unknown parameter: log_max_bin
[LightGBM] [Warning] Unknown parameter: log_max_bin
[LightGBM] [Info] Number of positive: 16775, number of negative: 45312
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6630
[LightGBM] [Info] Number of data points in the train set: 62087, number of used features: 26
[LightGBM] [Warning] Unknown parameter: log_max_bin
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270185 -> initscore=-0.993682
[LightGBM] [Info] Start training from score -0.993682
[1]	valid_0's binary_logloss: 0.540279
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.52001
[3]	valid_0's binary_logloss: 0.499769
[4]	valid_0's binary_logloss: 0.486583
[5]	valid_0's binary_logloss: 0.461765
[6]	valid_0's binary_logloss:

array([1., 0., 0., ..., 0., 0., 0.])

In [283]:
print(classification_report(np.rint(bst.predict(Best_Heckman_X)), Best_Heckman_y['AGN'], digits=4))
print(balanced_accuracy_score(np.rint(bst.predict(Best_Heckman_X)), Best_Heckman_y['AGN']))

              precision    recall  f1-score   support

         0.0     0.7085    0.5656    0.6291      3649
         1.0     0.8454    0.9108    0.8768      9513

    accuracy                         0.8151     13162
   macro avg     0.7769    0.7382    0.7530     13162
weighted avg     0.8074    0.8151    0.8081     13162

0.73819406292242


In [234]:
Best_Heckman_data = pd.read_csv("../../../Data/Best&Heckman/BestHeckman+SDSS+wise+LOFAR_better_fixed_fluxes.csv")
Best_Heckman_data['Source'] = 'BH'

In [235]:
# Only selecting data with a classification
Best_Heckman_data = Best_Heckman_data[Best_Heckman_data['Classification'] != 'Radio-loud AGN'] 

In [236]:
Best_Heckman_X = Best_Heckman_data[[c for c in Best_Heckman_data.columns if c != 'Classification']]
Best_Heckman_y = Best_Heckman_data[['Classification', 'Source']]

In [237]:
Best_Heckman_X = Best_Heckman_X[['Z_BEST', 'u_flux_corr',
       'g_flux_corr', 'R_flux_corr', 'I_flux_corr', 'z_flux_corr', 'ch1_flux_corr', 'ch2_flux_corr',
       'J_flux_corr', 'H_flux_corr', 'Ks_flux_corr', 'Peak_flux', 'Total_flux']]

# Adding nans to missing columns
Best_Heckman_X[['NUV_flux_corr', 'Bw_flux_corr', 'y_flux_corr', 'K_flux_corr', 
                'F_MIPS_24', 'F_PACS_100', 'F_PACS_160', 'F_SPIRE_250', 'F_SPIRE_350',
                'F_SPIRE_500', 'nb921_hsc_flux_corr', 'ch3_flux_corr', 'ch4_flux_corr', 'FUV_flux_corr']] = np.nan

In [238]:
Best_Heckman_y['AGN'] =  Best_Heckman_y.apply(AGN, axis=1, result_type='expand')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [244]:
Best_Heckman_X = Best_Heckman_X[features_num]

In [211]:
filt = ~Best_Heckman_X['Total_flux'].isna()

In [208]:
print(classification_report(model.predict(Best_Heckman_X), Best_Heckman_y['AGN'], digits=4))
print(balanced_accuracy_score(model.predict(Best_Heckman_X), Best_Heckman_y['AGN']))

              precision    recall  f1-score   support

           0     0.1593    0.2094    0.1809      2216
           1     0.8291    0.7763    0.8018     10946

    accuracy                         0.6808     13162
   macro avg     0.4942    0.4928    0.4914     13162
weighted avg     0.7163    0.6808    0.6973     13162

0.492825791991008
